In [27]:
from omrdatasettools import Downloader, OmrDataset
import xmlschema

import sys
import os
import glob
import re
from PIL import Image
from lxml import etree
from pprint import pprint
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.cluster import KMeans

from datasets import MuscimaObjects

In [2]:
muscimaroot = 'muscima'

dataset = MuscimaObjects.MuscimaObjects(muscimaroot)

In [38]:
label_list_all = dataset.label_list
label_list_alt = ['noteheadFull', 'noteheadHalf', 'noteheadWhole', 'accidentalSharp', 'accidentalFlat', 'accidentalNatural',
                    'gCflef', 'fClef', 'cClef']

dataset.label_list = label_list_alt

In [39]:
bboxes = []

image, target = dataset[0]

boxes = target['boxes'].numpy()

boxes_w_h = np.asarray([boxes[:,2] - boxes[:,0], boxes[:,3] - boxes[:,1]]).T

boxes_w_h.shape

all_boxes = np.ndarray(shape=(0,2))
for image, target in dataset:
    boxes = target['boxes'].numpy()
    boxes_w_h = np.asarray([boxes[:,2] - boxes[:,0], boxes[:,3] - boxes[:,1]]).T
    all_boxes = np.vstack((all_boxes, boxes_w_h))

In [40]:
all_boxes.shape

(27749, 2)

In [24]:
def average_iou(bboxes, anchors):
    """Calculates the Intersection over Union (IoU) between bounding boxes and
    anchors.

    Args:
    bboxes : Array of bounding boxes in [width, height] format.
    anchors : Array of aspect ratios [n, 2] format.

    Returns:
    avg_iou_perc : A Float value, average of IOU scores from each aspect ratio
    """
    intersection_width = np.minimum(anchors[:, [0]], bboxes[:, 0]).T
    intersection_height = np.minimum(anchors[:, [1]], bboxes[:, 1]).T

    if np.any(intersection_width == 0) or np.any(intersection_height == 0):
        raise ValueError("Some boxes have zero size.")

    intersection_area = intersection_width * intersection_height
    boxes_area = np.prod(bboxes, axis=1, keepdims=True)
    anchors_area = np.prod(anchors, axis=1, keepdims=True).T
    union_area = boxes_area + anchors_area - intersection_area
    avg_iou_perc = np.mean(np.max(intersection_area / union_area, axis=1)) * 100

    return avg_iou_perc

def kmeans_aspect_ratios(bboxes, kmeans_max_iter, num_aspect_ratios):
  """Calculate the centroid of bounding boxes clusters using Kmeans algorithm.

  Args:
  bboxes : Array of bounding boxes in [width, height] format.
  kmeans_max_iter : Maximum number of iterations to find centroids.
  num_aspect_ratios : Number of centroids to optimize kmeans.

  Returns:
  aspect_ratios : Centroids of cluster (optmised for dataset).
  avg_iou_prec : Average score of bboxes intersecting with new aspect ratios.
  """

  assert len(bboxes), "You must provide bounding boxes"

  normalized_bboxes = bboxes / np.sqrt(bboxes.prod(axis=1, keepdims=True))
  
  # Using kmeans to find centroids of the width/height clusters
  kmeans = KMeans(
      init='random', n_clusters=num_aspect_ratios, random_state=0, max_iter=kmeans_max_iter)
  kmeans.fit(X=normalized_bboxes)
  ar = kmeans.cluster_centers_

  assert len(ar), "Unable to find k-means centroid, try increasing kmeans_max_iter."

  avg_iou_perc = average_iou(normalized_bboxes, ar)

  if not np.isfinite(avg_iou_perc):
    sys.exit("Failed to get aspect ratios due to numerical errors in k-means")

  aspect_ratios = [w/h for w,h in ar]

  return aspect_ratios, avg_iou_perc

In [35]:
# Tune this based on your accuracy/speed goals as described above
num_aspect_ratios = 4 # can be [2,3,4,5,6]

# Tune the iterations based on the size and distribution of your dataset
# You can check avg_iou_prec every 100 iterations to see how centroids converge
kmeans_step = 2
kmeans_max_iter = 20

# These should match the training pipeline config ('fixed_shape_resizer' param)
width = 1024
height = 1024

# Get the ground-truth bounding boxes for our dataset
#bboxes = xml_to_boxes(path=XML_PATH, rescale_width=width, rescale_height=height)
#bboxes = xml_to_boxes(path=XML_PATH)

In [41]:
i = 1
while i*kmeans_step < kmeans_max_iter:
  aspect_ratios, avg_iou_perc =  kmeans_aspect_ratios(
                                      bboxes=all_boxes,
                                      kmeans_max_iter=i*kmeans_step,
                                      num_aspect_ratios=num_aspect_ratios)
  aspect_ratios = sorted(aspect_ratios)
  print('iteration:', str(i*kmeans_step))
  print('Aspect ratios generated:', [round(ar,2) for ar in aspect_ratios])
  print('Average IOU with anchors:', avg_iou_perc)
  i = i + 1



aspect_ratios = sorted(aspect_ratios)

print('Aspect ratios generated:', [round(ar,2) for ar in aspect_ratios])
print('Average IOU with anchors:', avg_iou_perc)

iteration: 2
Aspect ratios generated: [0.35, 0.62, 0.88, 1.21]
Average IOU with anchors: 91.0661043734289
iteration: 4
Aspect ratios generated: [0.34, 0.61, 0.88, 1.21]
Average IOU with anchors: 91.00637915771134
iteration: 6
Aspect ratios generated: [0.34, 0.61, 0.88, 1.21]
Average IOU with anchors: 90.98162161591621
iteration: 8
Aspect ratios generated: [0.34, 0.6, 0.88, 1.21]
Average IOU with anchors: 90.95950799557914
iteration: 10
Aspect ratios generated: [0.34, 0.6, 0.88, 1.21]
Average IOU with anchors: 90.95950799557914
iteration: 12
Aspect ratios generated: [0.34, 0.6, 0.88, 1.21]
Average IOU with anchors: 90.95950799557914
iteration: 14
Aspect ratios generated: [0.34, 0.6, 0.88, 1.21]
Average IOU with anchors: 90.95950799557914
iteration: 16
Aspect ratios generated: [0.34, 0.6, 0.88, 1.21]
Average IOU with anchors: 90.95950799557914
iteration: 18
Aspect ratios generated: [0.34, 0.6, 0.88, 1.21]
Average IOU with anchors: 90.95950799557914
Aspect ratios generated: [0.34, 0.6, 0.